## Import packages

In [1]:
import numpy as np
import tensorflow as tf

## Visualization
```
bokeh serve --show widget_gradient_descent.py
```

## Gradient Descent 1D

Suppose we have a one variable linear function $\mathbf{y}=3.145\mathbf{x}$, where $y$ and $x$ is a $m$ length vector. Now we would like to construct a model to fit the above-mentioned function: $\mathbf{\hat{y}}=\mathbf{x}w$. It is trivial to see that $w=3.145$ gives us the exact function of $\mathbf{y}$. Here, we will use a __gradient descent__ algorithm to approximate the solution.

![Gradient Descent 1D](assets/gradient_descent1d.gif)

In [2]:
num_samples = 20
tf.random.set_seed(0)
x = tf.random.normal((num_samples, 1), mean=0, stddev=1)
y = 3.145 * x

w = tf.Variable(0.)
y_hat = w * x

To do so, we defined the __objective function__ to be the square deviation between $\mathbf{y}$ and $\mathbf{\hat{y}}$:
$$
\begin{align}
L&=\sum\limits_{i=1}^{m}(y_i - \hat{y}_i)^2\\
 &=\sum\limits_{i=1}^{m}(y_i - wx_i)^2
\end{align}
$$
The objective functin can be written as $(\mathbf{y}-\mathbf{Xw})^TI(\mathbf{y}-\mathbf{Xw})$. From the previous section, we know that the gradient of $L$ with respect to $w$ is:
$$
\nabla_{\mathbf{w}}L(\mathbf{\hat{y}})=[-2(\mathbf{y}-\mathbf{\hat{y}})^T\mathbf{X}]^T
$$
Note that the direction of the gradient will always point way from the weight that yield local minimum of the loss surface, therefore, we can update the weight on the opposite direction of $\nabla_{\mathbf{w}}L(\mathbf{\hat{y}})$:
$$
\mathbf{w}^{(t+1)}=\mathbf{w}^{(t)} - \eta\nabla_{\mathbf{w}}L(\mathbf{\hat{y}})
$$
where $\eta$ is a hyperparameter called __learning rate__, which is used to control the magnitude of each updates.



In [3]:
eta = 0.03
for i in range(0, 10):
    with tf.GradientTape() as g:
        g.watch(w)
        loss = tf.reduce_sum(tf.pow(y - w * x, 2))
    gradient = g.gradient(loss, w)
    w = w.assign(w - eta * gradient) 
    print(f'iteration:{i+1:>2}    weight: {w.numpy():>.3f}    loss: {loss.numpy():>7.3f}')

iteration: 1    weight: 2.204    loss: 115.505
iteration: 2    weight: 2.863    loss:  10.349
iteration: 3    weight: 3.061    loss:   0.927
iteration: 4    weight: 3.120    loss:   0.083
iteration: 5    weight: 3.137    loss:   0.007
iteration: 6    weight: 3.143    loss:   0.001
iteration: 7    weight: 3.144    loss:   0.000
iteration: 8    weight: 3.145    loss:   0.000
iteration: 9    weight: 3.145    loss:   0.000
iteration:10    weight: 3.145    loss:   0.000


## Gradient Descent 2D
Now let's look into multiple variables regression:  $\mathbf{y}=3.145\mathbf{x}_1 + 6.323\mathbf{x}_2$, we whould like to find $\hat{y}=w_1\mathbf{x}_1+w_2\mathbf{x}_2$ that approximate this function:
$$
\mathbf{\hat{y}} = \mathbf{X}\mathbf{w} =
\begin{bmatrix}
    x_{1,1} & x_{1,2}\\
    x_{2,1} & x_{2,2}\\
    \vdots  & \vdots \\
    x_{m,1} & x_{m,2}\\
\end{bmatrix}\begin{bmatrix}
    w_{1}   \\
    w_{2}   \\
\end{bmatrix}
$$
The square loss function then become:
$$
\begin{align}
L&=\sum\limits_{i=1}^{m}(y_i - \hat{y}_i)^2\\
 &=\sum\limits_{i=1}^{m}\sum\limits_{j=1}^{2}(y_i - w_jx_{ij})^2
\end{align}
$$
The gradient of $L$ with respect to $\mathbf{w}$ remains the same, but with additional dimension:
$$
\nabla_{\mathbf{w}}L(\mathbf{\hat{y}})=[-2(\mathbf{y}-\mathbf{\hat{y}})^T\mathbf{X}]^T
$$
![Gradient Descent 2D](assets/gradient_descent2d.gif)

In [4]:

eta = 0.03
num_samples = 20

tf.random.set_seed(0)
x = tf.random.normal((num_samples, 2), mean=0, stddev=1)
y = tf.reshape(3.145 * x[:, 0] +  6.323 * x[:, 1], (-1, 1))

w = tf.Variable(tf.zeros((2, 1)))

for i in range(0, 10):
    with tf.GradientTape() as g:
        g.watch(w)
        loss = tf.reduce_sum(tf.pow(y - tf.matmul(x, w), 2))
    gradient = g.gradient(loss, w)
    w = w.assign(w - eta * gradient) 
    print(f'iteration:{i+1:>2}    weight 1: {w.numpy()[0, 0]:>.3f}    weight 2: {w.numpy()[1, 0]:>.3f}    loss: {loss.numpy():>7.3f}')

iteration: 1    weight 1: 4.542    weight 2: 4.901    loss: 754.581
iteration: 2    weight 1: 2.929    weight 2: 5.802    loss:  58.888
iteration: 3    weight 1: 3.249    weight 2: 6.202    loss:   4.620
iteration: 4    weight 1: 3.130    weight 2: 6.280    loss:   0.364
iteration: 5    weight 1: 3.153    weight 2: 6.313    loss:   0.029
iteration: 6    weight 1: 3.144    weight 2: 6.319    loss:   0.002
iteration: 7    weight 1: 3.146    weight 2: 6.322    loss:   0.000
iteration: 8    weight 1: 3.145    weight 2: 6.323    loss:   0.000
iteration: 9    weight 1: 3.145    weight 2: 6.323    loss:   0.000
iteration:10    weight 1: 3.145    weight 2: 6.323    loss:   0.000


## Epoch and Batch
When we compute the loss, we can either consider every sample or a subset of samples at a time. We call the iteration that every sample in the dataset has been used to update the parameters an __epoch__. In each epoch, we can split the samples into different __batches__. 
* __Mini-batch gradient descent__: updates the parameters consider a batch of samples at a time. 
* __Stochastic gradient descent (SGD)__: update the parameters consider one sample at a time).

In [5]:

def mini_batch_gradient_descent(x, y, w, num_epochs, batch_size, eta=0.03, example=1):
    num_samples = x.shape[0]

    for i in range(0, num_epochs):

        indices = tf.range(start=0, limit=num_samples, dtype=tf.int32)
        tf.random.set_seed(i)
        shuffled_indices = tf.random.shuffle(indices)

        shuffled_x = tf.gather(x, shuffled_indices)
        shuffled_y = tf.gather(y, shuffled_indices)

        batches = [batch_size] * (num_samples // batch_size)
        if num_samples % batch_size != 0:
            batches.extend([num_samples % batch_size])

        for j, (batch_x, batch_y) in enumerate(zip(tf.split(shuffled_x, batches, axis=0), tf.split(shuffled_y, batches, axis=0))):
            with tf.GradientTape() as g:
                g.watch(w)
                if example == 1:
                    loss = tf.reduce_sum(tf.pow(batch_y - tf.matmul(batch_x, w), 2))
                elif example == 2:
                    loss = tf.reduce_sum(tf.pow(batch_y - tf.matmul(batch_x, tf.pow(w, 2)), 2))
            gradient = g.gradient(loss, w)
            w = w.assign(w - eta * gradient) 
            print(f'epoch:{i+1:>2}    batch:{j+1:>2}     weight 1: {w.numpy()[0, 0]:>.3f}    weight 2: {w.numpy()[1, 0]:>.3f}    loss: {loss.numpy():>7.3f}')
    return


In [6]:
eta = 0.03
batch_size = 10
num_epochs = 10
num_samples = 20

tf.random.set_seed(0)
x = tf.random.normal((num_samples, 2), mean=0, stddev=1)
y = tf.reshape(3.145 * x[:, 0] +  6.323 * x[:, 1], (-1, 1))

w = tf.Variable(tf.zeros((2, 1)))

mini_batch_gradient_descent(x, y, w, num_epochs, batch_size, eta) 


epoch: 1    batch: 1     weight 1: 2.133    weight 2: 1.662    loss: 286.972
epoch: 1    batch: 2     weight 1: 3.116    weight 2: 3.946    loss: 194.002
epoch: 2    batch: 1     weight 1: 3.037    weight 2: 4.373    loss:  16.866
epoch: 2    batch: 2     weight 1: 3.350    weight 2: 5.456    loss:  35.755
epoch: 3    batch: 1     weight 1: 3.191    weight 2: 5.726    loss:   4.440
epoch: 3    batch: 2     weight 1: 3.236    weight 2: 5.972    loss:   2.423
epoch: 4    batch: 1     weight 1: 3.188    weight 2: 6.110    loss:   0.875
epoch: 4    batch: 2     weight 1: 3.176    weight 2: 6.177    loss:   0.248
epoch: 5    batch: 1     weight 1: 3.182    weight 2: 6.231    loss:   0.128
epoch: 5    batch: 2     weight 1: 3.149    weight 2: 6.263    loss:   0.070
epoch: 6    batch: 1     weight 1: 3.147    weight 2: 6.286    loss:   0.023
epoch: 6    batch: 2     weight 1: 3.149    weight 2: 6.298    loss:   0.008
epoch: 7    batch: 1     weight 1: 3.144    weight 2: 6.307    loss:   0.004

If the loss function is a convex function with respect to the parameters, gradient descent is guaranteed to find the optimal parameters that minimize the loss globally (if the learning rate is properly configured). However, this assumption does not always hold. For example, if we defined a non-linear transformation on $\mathbf{w}$: $\hat{y}=w_1^2\mathbf{x}_1+w_2^2\mathbf{x}_2$:
$$
\begin{align}
L&=\sum\limits_{i=1}^{m}(y_i - \hat{y}_i)^2\\
 &=\sum\limits_{i=1}^{m}\sum\limits_{j=1}^{2}(y_i - w_j^2x_{ij})^2
\end{align}
$$
The gradient of $L$ with respect to $w$ then become:
$$
\nabla_{\mathbf{w}}L(\hat{y})=-4(\mathbf{y} - \hat{y})^T\mathbf{X}\mathbf{w}
$$
The gradient descent optimization might not be able to find the optimal parameters that globally minimize the loss function.


In [7]:
eta = 1e-4
batch_size = 10
num_samples = 20

tf.random.set_seed(0)
x = tf.random.normal((num_samples, 2), mean=0, stddev=1)
y = tf.reshape(3.145 ** 2 * x[:, 0] +  6.323 ** 2 * x[:, 1], (-1, 1))

w = tf.Variable(tf.zeros((2, 1)))

mini_batch_gradient_descent(x, y, w, 10, batch_size, eta, example=2) 

epoch: 1    batch: 1     weight 1: 0.000    weight 2: 0.000    loss: 8055.223
epoch: 1    batch: 2     weight 1: 0.000    weight 2: 0.000    loss: 14630.254
epoch: 2    batch: 1     weight 1: 0.000    weight 2: 0.000    loss: 5639.693
epoch: 2    batch: 2     weight 1: 0.000    weight 2: 0.000    loss: 17045.783
epoch: 3    batch: 1     weight 1: 0.000    weight 2: 0.000    loss: 8882.165
epoch: 3    batch: 2     weight 1: 0.000    weight 2: 0.000    loss: 13803.312
epoch: 4    batch: 1     weight 1: 0.000    weight 2: 0.000    loss: 12194.377
epoch: 4    batch: 2     weight 1: 0.000    weight 2: 0.000    loss: 10491.100
epoch: 5    batch: 1     weight 1: 0.000    weight 2: 0.000    loss: 13091.315
epoch: 5    batch: 2     weight 1: 0.000    weight 2: 0.000    loss: 9594.162
epoch: 6    batch: 1     weight 1: 0.000    weight 2: 0.000    loss: 11150.024
epoch: 6    batch: 2     weight 1: 0.000    weight 2: 0.000    loss: 11535.453
epoch: 7    batch: 1     weight 1: 0.000    weight 2: 0.

We usually solve this by proper weight initialization or applying the optimizers, which we will explain in more details in other exercise:

In [8]:
eta = 1e-4
w = tf.Variable(tf.convert_to_tensor([[3.], [6.]]))

mini_batch_gradient_descent(x, y, w, 10, batch_size, eta, example=2) 

epoch: 1    batch: 1     weight 1: 3.013    weight 2: 6.041    loss:  77.747
epoch: 1    batch: 2     weight 1: 3.028    weight 2: 6.110    loss: 109.897
epoch: 2    batch: 1     weight 1: 3.039    weight 2: 6.128    loss:  25.780
epoch: 2    batch: 2     weight 1: 3.051    weight 2: 6.186    loss:  63.831
epoch: 3    batch: 1     weight 1: 3.058    weight 2: 6.207    loss:  17.849
epoch: 3    batch: 2     weight 1: 3.068    weight 2: 6.235    loss:  20.875
epoch: 4    batch: 1     weight 1: 3.075    weight 2: 6.254    loss:  11.259
epoch: 4    batch: 2     weight 1: 3.081    weight 2: 6.267    loss:   6.715
epoch: 5    batch: 1     weight 1: 3.087    weight 2: 6.281    loss:   5.718
epoch: 5    batch: 2     weight 1: 3.092    weight 2: 6.288    loss:   3.027
epoch: 6    batch: 1     weight 1: 3.097    weight 2: 6.295    loss:   2.363
epoch: 6    batch: 2     weight 1: 3.101    weight 2: 6.301    loss:   2.029
epoch: 7    batch: 1     weight 1: 3.106    weight 2: 6.305    loss:   1.479